Setup Kaggle and Dataset

In [8]:
!mkdir ~/.kaggle

In [10]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [11]:
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 71.9MB/s]


In [13]:
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


İmport Libary and create dataset

In [97]:
import pandas as pd
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler ,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import pickle

In [16]:
df = pd.read_csv('train.csv')

In [27]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [47]:
#Dont get NAN values
main_df = df[['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street','LotShape', 'LandContour','Utilities','MiscVal', 'MoSold', 'YrSold', 'SaleType','SalePrice']].dropna()

In [48]:
# Create X table
X=pd.get_dummies(main_df.drop('SalePrice', axis=1))
#Create target column
y = main_df.SalePrice

In [49]:
X.head()

,MSSubClass,LotFrontage,LotArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,...,Utilities_AllPub,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,60,65.0,8450,0,2,2008,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
1,20,80.0,9600,0,5,2007,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
2,60,68.0,11250,0,9,2008,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,70,60.0,9550,0,2,2006,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,60,84.0,14260,0,12,2008,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1


In [45]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [51]:
3#create pipline for algoritim
pipeline = make_pipeline(StandardScaler(), RandomForestRegressor())

In [52]:
pipeline.fit(X,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [53]:
pipeline.predict(X)

array([202843.32, 165708.75, 218506.  , ..., 227121.25, 148002.25,
       151492.  ])

In [66]:
with open('HouesPrice.pkl','wb') as f:
  pickle.dump(pipeline, f)

In [123]:
with open('HouesPrice.pkl','rb') as f:
  reloaded_model = pickle.load(f)

In [68]:
reloaded_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

## Create Columns transformers  for preprocresing pipline
Link to sklearn Column trasformer
[link text](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#)



In [82]:
#pipline class
custom_pipeline = Pipeline([('scaling', StandardScaler()), ('rfmodel', RandomForestRegressor())])
custom_pipeline

Pipeline(steps=[('scaling', StandardScaler()),
                ('rfmodel', RandomForestRegressor())])

In [84]:
#make_pipline class
make_pipeline_model = make_pipeline(StandardScaler(), RandomForestRegressor())
make_pipeline_model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [87]:
from sklearn.compose import ColumnTransformer

In [108]:
#Numeric Features 
numeric_features  = main_df.drop('SalePrice' ,axis = 1).select_dtypes(exclude = 'object').columns
numeric_pipeline = Pipeline([('scaling', StandardScaler())])

In [99]:
#Categorical  Features
categorical_features = main_df.select_dtypes('object').columns
categorical_pipeline = Pipeline([('onehot', OneHotEncoder())])

In [109]:
transformers = ColumnTransformer([('numeric_preprocresing', numeric_pipeline , numeric_features),
                                ('categorical_preprocresing', categorical_pipeline , categorical_features)
                                ])

In [110]:
transformers

ColumnTransformer(transformers=[('numeric_preprocresing',
                                 Pipeline(steps=[('scaling',
                                                  StandardScaler())]),
                                 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MiscVal', 'MoSold', 'YrSold'], dtype='object')),
                                ('categorical_preprocresing',
                                 Pipeline(steps=[('onehot', OneHotEncoder())]),
                                 Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'SaleType'],
      dtype='object'))])

In [111]:
ml_pipeline = Pipeline([('AlL_Column_Preprocressing', transformers), ('randomforestregresion',RandomForestRegressor())])
ml_pipeline

Pipeline(steps=[('AlL_Column_Preprocressing',
                 ColumnTransformer(transformers=[('numeric_preprocresing',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MiscVal', 'MoSold', 'YrSold'], dtype='object')),
                                                 ('categorical_preprocresing',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'SaleType'],
      dtype='object'))])),
                ('randomforestregresion', RandomForestRegressor())])

In [115]:
X= main_df.drop('SalePrice', axis = 1)
y = main_df['SalePrice']

In [117]:
ml_pipeline.fit(X,y)

Pipeline(steps=[('AlL_Column_Preprocressing',
                 ColumnTransformer(transformers=[('numeric_preprocresing',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MiscVal', 'MoSold', 'YrSold'], dtype='object')),
                                                 ('categorical_preprocresing',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'SaleType'],
      dtype='object'))])),
                ('randomforestregresion', RandomForestRegressor())])

In [127]:
ml_pipeline.predict(X)

array([207330.  , 169644.32, 217740.86, ..., 225373.07, 148661.25,
       153047.  ])

In [128]:
with open('ColumnTranformer.pkl','wb') as f:
  pickle.dump(ml_pipeline, f)

In [129]:
with open('ColumnTranformer.pkl','rb') as f:
  reloaded_ml_model = pickle.load(f)

In [130]:
reloaded_ml_model

Pipeline(steps=[('AlL_Column_Preprocressing',
                 ColumnTransformer(transformers=[('numeric_preprocresing',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MiscVal', 'MoSold', 'YrSold'], dtype='object')),
                                                 ('categorical_preprocresing',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'SaleType'],
      dtype='object'))])),
                ('randomforestregresion', RandomForestRegressor())])